# Import required libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.cm as cm
import scipy.stats
from matplotlib import pyplot as plt 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.layers import BatchNormalization
from tensorflow.keras import regularizers


# prevent tensorflow from allocating the entire GPU memory at once
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os
import sys
os.chdir('/content/drive/MyDrive/CrowdLayer-R')
cwd = os.getcwd()
sys.path.append("/content/drive/MyDrive/CrowdLayer-R/crowd_layer")
# packages for learning from crowds
from crowd_layer.crowd_layers import CrowdsClassification, MaskedMultiCrossEntropy
from crowd_layer.crowd_aggregators import CrowdsCategoricalAggregator

In [ ]:
train_data = tf.data.experimental.load('/content/drive/Shareddrives/Multiple Anotators/CrowdLayer/Data/cats_dogs_Tr')
Test_data = tf.data.experimental.load('/content/drive/Shareddrives/Multiple Anotators/CrowdLayer/Data/cats_dogs_Te')

In [ ]:
image_count_train = tf.data.experimental.cardinality(train_data).numpy()
image_count_train

In [ ]:
image_count_test = tf.data.experimental.cardinality(Test_data).numpy()
image_count_test

In [ ]:
subset = 18610
train_data = train_data.take(18610)
image_count1 = tf.data.experimental.cardinality(train_data).numpy()
image_count1

In [ ]:
image_count = 3000
 
val_size = int(image_count1 * 0.25)
train_ds = train_data.skip(val_size)
val_ds  = train_data.take(val_size)

In [ ]:
image_count_val = tf.data.experimental.cardinality(val_ds).numpy()
image_count_val

In [ ]:
image_count_Tr = tf.data.experimental.cardinality(train_ds).numpy()
image_count_Tr

In [ ]:
batch_size = 150
train_batches= train_ds.shuffle(1024).batch(batch_size)
val_batches = val_ds.shuffle(1024).batch(50)
test_batches = Test_data.shuffle(1024).batch(50)

In [ ]:
classNames = ['cat', 'dog']


i = 0

fig, ax = plt.subplots(1, 7, figsize=(15,15))
 
for image,label in train_ds.take(7):
 #  print(image.shape)
 #   predictedLabel = int(predictions[i] >= 0.5)
   # print(label)
   ax[i].axis('off')
   ax[i].set_title(classNames[label.numpy()])
   ax[i].imshow(image)
    
   i += 1
 
# plt.show()

# Configuration parameters

In [ ]:
# DATA_PATH = "/home/fmpr/datasets/deep-crowds-datasets/dogs_vs_cats_sim/prepared/"
N_CLASSES = 2
BATCH_SIZE = 128
N_EPOCHS = 60

# Load data

In [ ]:
# def load_data(filename):
#     f = open(filename)
#     data = np.load(f)
#     f.close()
#     return data

In [ ]:
# data_train_list = []
# labels_train_list = []

# for data_train1, labels_train1 in train_ds:
#      data_train_list.append(data_train1)
#      labels_train_list.append(labels_train1)

# data_train =  np.array(data_train_list)
# labels_train = np.array(labels_train_list)
# print(data_train.shape)
# print(labels_train.shape)
# del data_train_list
# del labels_train_list

In [ ]:
# data_val_list = []
# labels_val_list = []

# for data_val1, labels_val1 in val_ds:
#      data_val_list.append(data_val1)
#      labels_val_list.append(labels_val1)

# data_val =  np.array(data_val_list)
# labels_val = np.array(labels_val_list)
# print(data_val.shape)
# print(labels_val.shape)
# del data_val_list
# del labels_val_list

In [ ]:
# data_test_list = []
# labels_test_list = []

# for data_test1, labels_test1 in Test_data:
#      data_test_list.append(data_test1)
#      labels_test_list.append(labels_test1)

# data_test =  np.array(data_test_list)
# labels_test = np.array(labels_test_list)
# print(data_test.shape)
# print(labels_test.shape)
# del data_test_list
# del labels_test_list

# Convert data to one-hot encoding

In [ ]:
# def one_hot(target, n_classes):
#     targets = np.array([target]).reshape(-1)
#     one_hot_targets = np.eye(n_classes)[targets]
#     return one_hot_targets

In [ ]:
# print("\nConverting to one-hot encoding...")
# labels_train_bin = one_hot(labels_train, N_CLASSES)
# print(labels_train_bin.shape)
# # labels_train_mv_bin = one_hot(labels_train_mv, N_CLASSES)
# # print labels_train_mv_bin.shape
# # labels_train_ds_bin = one_hot(labels_train_ds, N_CLASSES)
# # print labels_train_ds_bin.shape
# labels_test_bin = one_hot(labels_test, N_CLASSES)
# print(labels_test_bin.shape)
# labels_val_bin = one_hot(labels_val, N_CLASSES)
# print(labels_val_bin.shape)

# answers_bin_missings = []
# for i in range(len(answers)):
#     row = []
#     for r in range(N_ANNOT):
#         if answers[i,r] == -1:
#             row.append(-1 * np.ones(N_CLASSES))
#         else:
#             row.append(one_hot(answers[i,r], N_CLASSES)[0,:])
#     answers_bin_missings.append(row)
# answers_bin_missings = np.array(answers_bin_missings).swapaxes(1,2)
# answers_bin_missings.shape

# Define the base deep learning model

Here we shall use features representation produced by the VGG16 network as the input. Our base model is then simply composed by one densely-connected layer with 128 hidden units and an output dense layer. We use 50% dropout between the two dense layers.

In [ ]:
def build_base_model():
    # base_model = Sequential()


    # base_model.add( Conv2D(32, (3, 3), activation='relu',
    # input_shape=(150, 150, 3)))
    # base_model.add( MaxPooling2D((2, 2)))
    # base_model.add( Dropout(0.2))

    # base_model.add( Conv2D(64, (3, 3), activation='relu'))
    # base_model.add( MaxPooling2D((2, 2)))
    # base_model.add( Dropout(0.2))

    # base_model.add( Conv2D(128, (3, 3), activation='relu'))
    # base_model.add( MaxPooling2D((2, 2)))
    # base_model.add(Dropout(0.2))

    # base_model.add(Conv2D(256, (3, 3), activation='relu'))
    # base_model.add(MaxPooling2D((2, 2)))
    # base_model.add(Dropout(0.2))

    # base_model.add(Flatten())


    # base_model.add(Dense(256, activation='relu'))
    # base_model.add(Dense(1, activation='sigmoid'))

  
    base_model = Sequential()
    base_model.add(BatchNormalization())
    base_model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    base_model.add(Activation('relu'))
    base_model.add(BatchNormalization())
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
   # base_model.add(Dropout(0.2))
   
    base_model.add(BatchNormalization())
    base_model.add(Conv2D(32, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(BatchNormalization())
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
   # base_model.add(Dropout(0.2))
    

    base_model.add(BatchNormalization())
    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(BatchNormalization())
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
   # base_model.add(Dropout(0.2))

    base_model.add(BatchNormalization())
    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(BatchNormalization())
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
   # base_model.add(Dropout(0.2))


    base_model.add(Flatten()) 
    base_model.add(BatchNormalization())
    base_model.add(Dense(128))
    base_model.add(Activation('relu'))
    base_model.add(BatchNormalization())
    base_model.add(Dropout(0.2))
    base_model.add(Dense(N_CLASSES))
    base_model.add(Activation('softmax'))

    base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm = 1.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return base_model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model1.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

# Auxiliary function for evaluating the models

In [ ]:
def eval_model(model, test_data, test_labels):
    # testset accuracy
    preds_test = model.predict(test_data)
    preds_test_num = np.argmax(preds_test, axis=1)
    accuracy_test = 1.0*np.sum(preds_test_num == test_labels) / len(test_labels)

    return accuracy_test

# Train the model on the true labels (ground truth) and evaluate on testset

In [ ]:
model = build_base_model()

# model = models.Sequential()


# model.add(layers.Conv2D(32, (3, 3), activation='relu',
# input_shape=(128, 128, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))

# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))

# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))

# model.add(layers.Flatten())


# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.summary()

In [ ]:

from sklearn.metrics import classification_report
l1 =   0.001
NUM_RUNS =10
ACC = np.zeros(NUM_RUNS)
AUC = np.zeros(NUM_RUNS)
for i in range(NUM_RUNS): #10
    print("iteration: " + str(i))
   
    trainHistory = model.fit(train_batches, validation_data= val_batches, epochs=N_EPOCHS,  callbacks=callbacks)
    # Create the history figure
    plt.figure(figsize=(16,9))
    for i in  trainHistory.history:
        plt.plot( trainHistory.history[i],label=i)
    plt.title('Model history')
    plt.legend()
    plt.grid()
  
    #Accuracy
    pred_2 = model.predict(test_batches)
    #model.evaluate(test_batches)
 
    report = classification_report( pred_2.argmax(axis=1),Y_true_test.ravel(),output_dict=True)
    ACC[i] = report['accuracy']
    #AUC
    val_AUC_metric = tf.keras.metrics.AUC(num_thresholds=3, from_logits = True)  
    val_logits =MA.predict(X_test) # model(X_test, training=False)
    # tf.print(y_batch_val)
    val_AUC_metric.update_state(Y_true_test, val_logits.argmax(axis=1).astype('float'))

    val_AUC = val_AUC_metric.result()
    val_AUC_metric.reset_states()
    val_AUC = val_AUC.numpy()
    print("Validation aUc: %.4f" % (float(val_AUC),))
    AUC[i] = val_AUC

In [ ]:
SS

In [ ]:

AUC

In [ ]:
print('Average Accuracy: ', np.round( ACC.mean(),4)*100) 
print('Average std: ',np.round(np.std( ACC),4)*100)

In [ ]:
print('Average Accuracy: ', np.round( AUC.mean(),4)*100) 
print('Average std: ',np.round(np.std( AUC),4)*100)

In [ ]:
# accuracy_test = eval_model(model, data_test, labels_test)
# print("Accuracy: Test: %.3f" % (accuracy_test,))